In [1]:

from jgtpy import JGTIDS as ids,JGTIDSRequest as RQ
from jgtpy import jgtapyhelper as tah
from jgtpy import JGTPDSP as pds
from jgtutils.jgtconstants import VOLUME



Here is the function I Coded myself to Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics in order to validate and learn if the usage of the SQUAT in the last observed wave could help algorithm in decision to trade or not when we have trading signals.

```python

# MFI Constants for the column logics
from jgtutils.jgtconstants import MFI
MFI_SQUAT = "mfi_sq"
MFI_GREEN = "mfi_green"
MFI_FADE = "mfi_fade"
MFI_FAKE = "mfi_fake"
MFI_SIGNAL = "mfi_sig"
MFI_VAL = "mfiv"




def _ids_add_mfi_squat_column_logics_v1(dfsrc, quiet=False):
    """
    Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics to the given DataFrame.

    Parameters:
    - dfsrc (DataFrame): The input DataFrame to which the MFI squat column logics will be added.
    - quiet (bool): If True, suppresses the output messages. Default is False.

    Returns:
    - dfsrc (DataFrame): The updated DataFrame with the MFI squat column logics added.
    """

    dfsrc[MFI_SQUAT] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_GREEN] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FADE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FAKE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
        ).astype(int)
    dfsrc[MFI_SIGNAL] = dfsrc.apply(
         lambda row: 4 if row[MFI_SQUAT] > 0 else 1 if row[MFI_GREEN] > 0 else 2 if row[MFI_FADE]>0 else 3 if row[MFI_FAKE]>0 else 0 , axis=1
     )
    dfsrc[MFI_VAL] = dfsrc.apply(
         lambda row: "++" if row[MFI_SIGNAL] == 1 else "--" if row[MFI_SIGNAL] == 2 else "-+" if row[MFI_SIGNAL] == 3 else "+-" if row[MFI_SIGNAL] == 4 else "0" , axis=1)
    return dfsrc





```
## Usage:

```python

from jgtpy import JGTIDSRequest as RQ
from jgtpy import jgtapyhelper as tah
from jgtpy import JGTPDSP as pds

## Get data
i="SPX500";t="D1"
dfsrc = pds.getPH(i,t,use_fresh=False)

#### Create our current Indicators


rq = RQ.JGTIDSRequest(i,t)
res_tah= tah.toids(dfsrc,rq=rq)


df_res_tah_with_mfi_sig=_ids_add_mfi_squat_column_logics_v1(res_tah)

```


You analyze the code, learn from it, suggest best practices or approve it and produce abstract, documentations etc to explain it.


### PURPOSE: Adequate Migration of IDS to TAH


## Get data

In [2]:

i="SPX500";t="D1"


In [3]:
# Compare the two datasets from ids.tocds(... ) and tah.toids(...)
## -->  MAKE A PLAN

In [4]:
dfsrc = pds.getPH(i,t,use_fresh=False)

In [5]:
dfsrc.head(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,High,Low,Close,Median
Date,,,,,,,,,,,,,,
2022-10-03 21:00:00,3682.96,3796.43,3674.16,3791.17,3683.36,3797.03,3674.76,3791.77,172013,3683.16,3796.73,3674.46,3791.47,3735.595
2022-10-04 21:00:00,3791.17,3806.85,3721.58,3782.65,3791.77,3807.35,3722.08,3783.05,162267,3791.47,3807.10,3721.83,3782.85,3764.465


In [6]:
dfsrc.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,High,Low,Close,Median
Date,,,,,,,,,,,,,,
2024-06-12 21:00:00,5433.32,5447.68,5403.46,5431.18,5433.72,5448.28,5403.96,5431.78,102582,5433.52,5447.98,5403.71,5431.48,5425.845
2024-06-13 21:00:00,5431.18,5432.31,5426.55,5430.93,5431.78,5432.91,5427.15,5431.53,1018,5431.48,5432.61,5426.85,5431.23,5429.730


#### Create our current Indicators

In [7]:

rq = RQ.JGTIDSRequest(i,t)


In [8]:

res_jgtids = ids.tocds(dfsrc,rq=rq)


In [9]:

# MFI Constants for the column logics
from jgtutils.jgtconstants import MFI
MFI_SQUAT = "mfi_sq"
MFI_GREEN = "mfi_green"
MFI_FADE = "mfi_fade"
MFI_FAKE = "mfi_fake"
MFI_SIGNAL = "mfi_sig"
MFI_VAL = "mfi_str"


In [10]:


def _cds_add_mfi_squat_column_logics_v1(dfsrc, quiet=False):
    """
    Adds MFI (Market Facilitation Index (reference: B. Williams)) squat column logics to the given DataFrame.

    Parameters:
    - dfsrc (DataFrame): The input DataFrame to which the MFI squat column logics will be added.
    - quiet (bool): If True, suppresses the output messages. Default is False.

    Returns:
    - dfsrc (DataFrame): The updated DataFrame with the MFI squat column logics added.
    """

    dfsrc[MFI_SQUAT] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_GREEN] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume > dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FADE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi < dfsrc.mfi.shift())
    ).astype(int)
    dfsrc[MFI_FAKE] = (
        (dfsrc.Volume > 0)
        & (dfsrc.Volume < dfsrc.Volume.shift())
        & (dfsrc.mfi > dfsrc.mfi.shift())
        ).astype(int)
    dfsrc[MFI_SIGNAL] = dfsrc.apply(
         lambda row: 4 if row[MFI_SQUAT] > 0 else 1 if row[MFI_GREEN] > 0 else 2 if row[MFI_FADE]>0 else 3 if row[MFI_FAKE]>0 else 0 , axis=1
     )
    dfsrc[MFI_VAL] = dfsrc.apply(
         lambda row: "++" if row[MFI_SIGNAL] == 1 else "--" if row[MFI_SIGNAL] == 2 else "-+" if row[MFI_SIGNAL] == 3 else "+-" if row[MFI_SIGNAL] == 4 else "0" , axis=1)
    return dfsrc



In [11]:
with_squat=_cds_add_mfi_squat_column_logics_v1(res_jgtids).copy()

In [12]:
with_squat.tail(2)


,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-12 21:00:00,5433.32,5447.68,5403.46,5431.18,5433.72,5448.28,5403.96,5431.78,102582,5433.52,...,0,0,0,0,0,0,1,0,2,--
2024-06-13 21:00:00,5431.18,5432.31,5426.55,5430.93,5431.78,5432.91,5427.15,5431.53,1018,5431.48,...,0,0,0,0,0,0,0,1,3,-+


In [13]:

col_selected=[VOLUME, MFI_VAL, MFI, MFI_SQUAT, MFI_GREEN, MFI_FADE, MFI_FAKE, MFI_SIGNAL]
observed_mfi_n_sig=with_squat[col_selected]

In [14]:
observed_mfi_n_sig

,Volume,mfi_str,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig
Date,,,,,,,,
2023-04-25 21:00:00,134679,0,35.075996,0,0,0,0,0
2023-04-26 21:00:00,104987,-+,81.438654,0,0,0,1,3
2023-04-27 21:00:00,123683,+-,50.144321,1,0,0,0,4
2023-04-30 21:00:00,93349,--,29.727153,0,0,1,0,2
2023-05-01 21:00:00,123371,++,69.724652,0,1,0,0,1
...,...,...,...,...,...,...,...,...
2024-06-09 21:00:00,88209,-+,42.251924,0,0,0,1,3
2024-06-10 21:00:00,83933,-+,61.656321,0,0,0,1,3
2024-06-11 21:00:00,135364,+-,55.036790,1,0,0,0,4


## BELLOW CONFIRM What is Expected


* JGTIDS (which is more than just IDS) has 60 columns
* TAH (jgtapyhelper) has 41 columns, just IDS

#### In summary

* JGTIDS with ids.tocds(...) GENERATES SIGNALS

----


#### TOCDS (JGTIDS) Signal Columns

fdbb
fdbs
fdb
aoaz
aobz
zlc
zlcb
zlcs
zcol
sz
bz
acs
acb
ss
sb
price_peak_above
price_peak_bellow
ao_peak_above
ao_peak_bellow

----


In [15]:
res_tah= tah.toids(dfsrc,rq=rq)

 ADDING SQUAT PROTO


In [16]:


df_res_tah_with_mfi_sig=_cds_add_mfi_squat_column_logics_v1(res_tah)
res_tah_with_mfi_sig=df_res_tah_with_mfi_sig.copy()
observed_mfi_n_sig_tah=res_tah_with_mfi_sig[col_selected]


In [17]:

observed_mfi_n_sig_tah.tail(2)


,Volume,mfi_str,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig
Date,,,,,,,,
2024-06-12 21:00:00,102582,--,43.155719,0,0,1,0,2
2024-06-13 21:00:00,1018,-+,565.815324,0,0,0,1,3


In [18]:
res_tah.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,fl55,fh89,fl89,mfi,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-12 21:00:00,5433.32,5447.68,5403.46,5431.18,5433.72,5448.28,5403.96,5431.78,102582,5433.52,...,0,0,0,43.155719,0,0,1,0,2,--
2024-06-13 21:00:00,5431.18,5432.31,5426.55,5430.93,5431.78,5432.91,5427.15,5431.53,1018,5431.48,...,0,0,0,565.815324,0,0,0,1,3,-+


In [19]:
res_jgtids.tail(2)

,BidOpen,BidHigh,BidLow,BidClose,AskOpen,AskHigh,AskLow,AskClose,Volume,Open,...,price_peak_above,price_peak_bellow,ao_peak_above,ao_peak_bellow,mfi_sq,mfi_green,mfi_fade,mfi_fake,mfi_sig,mfi_str
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-12 21:00:00,5433.32,5447.68,5403.46,5431.18,5433.72,5448.28,5403.96,5431.78,102582,5433.52,...,0,0,0,0,0,0,1,0,2,--
2024-06-13 21:00:00,5431.18,5432.31,5426.55,5430.93,5431.78,5432.91,5427.15,5431.53,1018,5431.48,...,0,0,0,0,0,0,0,1,3,-+


In [20]:
res_jgtids.columns


Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5', 'fh8', 'fl8', 'fh13', 'fl13',
       'fh21', 'fl21', 'fh34', 'fl34', 'fh55', 'fl55', 'fh89', 'fl89', 'mfi',
       'fdbb', 'fdbs', 'fdb', 'aoaz', 'aobz', 'zlc', 'zlcb', 'zlcs', 'zcol',
       'sz', 'bz', 'acs', 'acb', 'ss', 'sb', 'price_peak_above',
       'price_peak_bellow', 'ao_peak_above', 'ao_peak_bellow', 'mfi_sq',
       'mfi_green', 'mfi_fade', 'mfi_fake', 'mfi_sig', 'mfi_str'],
      dtype='object')

In [21]:
res_tah.columns

Index(['BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh',
       'AskLow', 'AskClose', 'Volume', 'Open', 'High', 'Low', 'Close',
       'Median', 'ao', 'ac', 'jaw', 'teeth', 'lips', 'bjaw', 'bteeth', 'blips',
       'fh', 'fl', 'fh3', 'fl3', 'fh5', 'fl5', 'fh8', 'fl8', 'fh13', 'fl13',
       'fh21', 'fl21', 'fh34', 'fl34', 'fh55', 'fl55', 'fh89', 'fl89', 'mfi',
       'mfi_sq', 'mfi_green', 'mfi_fade', 'mfi_fake', 'mfi_sig', 'mfi_str'],
      dtype='object')

In [22]:

# Compare the two datasets from ids.tocds(... ) and tah.toids(...)
## List of columns that are different
for c in res_jgtids.columns:
    if c not in res_tah.columns:
        print(c)


fdbb
fdbs
fdb
aoaz
aobz
zlc
zlcb
zlcs
zcol
sz
bz
acs
acb
ss
sb
price_peak_above
price_peak_bellow
ao_peak_above
ao_peak_bellow
